### 没参数的装饰器

装饰器还有更大的灵活性，例如带参数的装饰器，在上面的装饰器调用中，该装饰器接收唯一的参数就是执行业务的函数 foo 。装饰器的语法允许我们在调用时，提供其它参数，比如@decorator(a)。这样，就为装饰器的编写和使用提供了更大的灵活性。比如，我们可以在装饰器中指定日志的等级，因为不同业务函数可能需要的日志级别是不一样的。

参考：https://foofish.net/python-decorator.html

#### 不带参数的装饰器

In [9]:
def use_logging(func):

    def wrapper():
        logging.warning("%s is running" % func.__name__)
        return func()
    return wrapper

@use_logging
def foo():
    print("i am foo")

foo()

i am foo


#### 带参数的装饰器
注意比"不带参数的装饰器"又多了一层, *感觉*这一层的作用就是提供额外的参数。  
如果里面那层再使用`nonlocal`引用最外层的参数 ？？？
> 上面的 use_logging 是允许带参数的装饰器。它<span class="girk">实际上是对原有装饰器的一个函数封装，并返回一个装饰器</span>。我们可以将它理解为一个含有参数的闭包。

In [12]:
def use_logging(level):
    
    def decorator(func):
        def wrapper(*args, **kwargs):
            if level == "warn":
                logging.warning("%s is running" % func.__name__)
            elif level == "info":
                logging.info("%s is running" % func.__name__)
            return func(*args)
        return wrapper

    return decorator


@use_logging(level="warn")
def foo(name='foo'):
    print("i am %s" % name)

foo()

i am foo


#### 类装饰器

相比函数装饰器，类装饰器具有灵活度大、高内聚、封装性等优点。<span class="girk">使用类装饰器主要依靠类的`__call__`方法</span>，当使用 @ 形式将装饰器附加到函数上时，就会调用此方法。

In [14]:
class Foo(object):
    def __init__(self, func):
        self._func = func

    def __call__(self):
        print ('class decorator runing')
        self._func()
        print ('class decorator ending')

@Foo
def bar():
    print ('bar')

bar()

class decorator runing
bar
class decorator ending


#### 装饰器顺序
一个函数还可以同时定义多个装饰器, 它的执行顺序是从里到外，最先调用最里层的装饰器，最后调用最外层的装饰器

## 9.5 可自定义属性的装饰器

你想写一个装饰器来包装一个函数,并且<span class="mark">允许用户提供参数在运行时控制装饰器行为。</span>

解决方案
<span class="mark">引入一个访问函数,使用 nonlocal 来修改内部变量。然后这个访问函数被作为一个属性赋值给包装函数.</span>

In [17]:
from functools import wraps, partial
import logging

# Utility decorator to attach a function as an attribute of obj
def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj)
    setattr(obj, func.__name__, func)
    return func


# 上面的偏函数有毛用？？
def logged(level, name=None, message=None):
    '''
    Add logging to a function. level is the logging
    level, name is the logger name, and message is the
    log message. If name and message aren't specified,
    they default to the function's module and name.
    '''
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        
        # attach setter functions
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel

        @attach_wrapper(wrapper)
        def set_msg(newlogmsg):
            nonlocal logmsg
            logmsg = newlogmsg
        
        return wrapper
    
    return decorate


# example use
@logged(logging.DEBUG)
def add(x, y):
    return x+y

# example use
@logged(logging.CRITICAL, name='example')
def spam():
    return "Spam!"

In [6]:
import logging
logging.basicConfig(level=logging.DEBUG)
add(2,3)

DEBUG:__main__:add


5

In [7]:
# change the log message
add.set_msg('Add called')
add(2,3)

DEBUG:__main__:Add called


5